In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 26.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 47.2 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=2362b06469956d921f420fb4f9c8bbc9ac62c222f86414f74d9e7c0262a45a26
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=5b8eeb89fa5f3b007868deef497c4e511ee0229072e68c1c73e300efdf17a6cb
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9

In [4]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/kaggle/input/clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet, 'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [5]:
test_dataset

,text,labels
0,Study shows that most Thais spend majority of ...,False
1,"Xbox One's Terraria Facing This ""Serious Issue""",True
2,The Simple Way To Banish Stinky Garlic Breath,True
3,He Went To Rip Weeds From His Lawn. Hours Late...,False
4,The Dish You Should Never Order in Rome,True
...,...,...
250,"Dog Dies One Hour After Hiking With His Owner,...",False
251,This is what happens when you leave a hotel cl...,False
252,This Texas GOP elector announces that he won't...,True
253,WikiLeaks' Julian Assange Reported Dead,False


In [6]:
train_dataset

,text,labels
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [7]:
validation_dataset

,text,labels
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
397,Target impresses with new designer collaboration,True
398,Research Reveals Indian Men Are Becoming Infer...,False
399,Jennifer Lawrence Knows Exactly What She Would...,True
400,Hillary Clinton Is Advertising in Deep-Red Neb...,False


# Training

In [8]:
import sklearn

args = {
    "batch_size": 8,
    "overwrite_output_dir": True,
    "num_train_epochs": 3,
    "fp16": False,
    "gradient_accumulation_steps": 4,
    "evaluate_during_training": True,
    "max_seq_length": 64,
    "learning_rate": 2e-5,
    "early_stopping_consider_epochs": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "acc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 3,
    "evaluate_during_training_steps": 331,
    "output_dir": "outputs/",
    'warmup_ratio': 0.06,
    # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
    #                    # <> überschreiben sich gegenseitig
    'save_steps': 2000,
    "manual_seed": 12345
}

In [ ]:
# configurations = []

# for learn_rate in [1e-5, 4e-5, 1e-4]:
#     for warumup_ratio in [0.02, 0.06, 0.1]:
#         for batch_size in [8, 16, 32]:
#             configurations += [{
#                 "overwrite_output_dir": True,
#                 "num_train_epochs": 2,
#                 "fp16": False,
#                 "train_batch_size": batch_size,
#                 "gradient_accumulation_steps": 4,
#                 "evaluate_during_training": True,
#                 "max_seq_length": 64,
#                 "learning_rate": learn_rate,
#                 "early_stopping_consider_epochs": True,
#                 "early_stopping_delta": 0.01,
#                 "early_stopping_metric": "acc",
#                 "early_stopping_metric_minimize": False,
#                 "early_stopping_patience": 3,
#                 "evaluate_during_training_steps": 331,
#                 "output_dir": "outputs/",
#                 'warmup_ratio': warumup_ratio,
#                 # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
#                 #                    # <> überschreiben sich gegenseitig
#                 'save_steps': 2000,
#                 "manual_seed": 12345
#             }]

In [9]:
from simpletransformers.classification import ClassificationModel

In [ ]:
# def train_model(config, num):
    
#     config["output_dir"] = "outputs/roberta_" + str(num)
    
#     model = ClassificationModel("roberta", "roberta-large", args = config)
#     # model.train_model(train_dataset)
#     model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [ ]:
# for config, num in zip(configurations, range(len(configurations))):
#     print('Running configuration number', num)
    
#     train_model(config, num)

In [40]:
!rm -r outputs/

In [10]:
model = ClassificationModel("roberta", "roberta-large", args = args)
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Running Epoch 1 of 3:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/402 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/402 [00:00<?, ?it/s]

(246,
 defaultdict(list,
             {'global_step': [82, 164, 246],
              'train_loss': [0.6357182264328003,
               0.39680802822113037,
               0.21878445148468018],
              'mcc': [0.3611531224634329,
               0.5912841752365663,
               0.6233277453144369],
              'tp': [111, 156, 164],
              'tn': [158, 163, 162],
              'fp': [36, 31, 32],
              'fn': [97, 52, 44],
              'auroc': [0.7551794210943695,
               0.8565870340999208,
               0.8697710150674068],
              'auprc': [0.7687749646416843,
               0.8695213923538325,
               0.8871535602034064],
              'acc': [0.6691542288557214,
               0.7935323383084577,
               0.8109452736318408],
              'eval_loss': [0.6177896179404914,
               0.46536434894683315,
               0.4472336135074204]}))

In [11]:
model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/402 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/51 [00:00<?, ?it/s]

({'mcc': 0.6233277453144369,
  'tp': 164,
  'tn': 162,
  'fp': 32,
  'fn': 44,
  'auroc': 0.8697710150674068,
  'auprc': 0.8871535602034064,
  'acc': 0.8109452736318408,
  'eval_loss': 0.4472336135074204},
 array([[ 3.30184847e-01, -5.08067787e-01],
        [-1.98620689e+00,  1.80565453e+00],
        [ 6.63025320e-01, -5.49846888e-01],
        [-1.83685827e+00,  2.09244490e+00],
        [-1.20446646e+00,  1.20617032e+00],
        [ 7.96036243e-01, -4.52510029e-01],
        [ 8.80807221e-01, -9.15490866e-01],
        [ 9.25301135e-01, -6.36689484e-01],
        [-1.63545835e+00,  2.17513251e+00],
        [ 7.75087357e-01, -9.12937105e-01],
        [ 1.84021056e-01, -4.22659904e-01],
        [ 1.08355582e+00, -1.32830524e+00],
        [ 1.76416233e-01,  7.39188269e-02],
        [ 4.73398566e-01, -3.30454260e-01],
        [ 5.27792454e-01, -1.57161981e-01],
        [-3.46252412e-01,  1.47706494e-01],
        [ 1.03033185e+00, -1.33325064e+00],
        [ 7.41859257e-01, -1.00774336e+00],
  

In [12]:
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/255 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

({'mcc': 0.49048724817211997,
  'tp': 92,
  'tn': 98,
  'fp': 30,
  'fn': 35,
  'auroc': 0.8281865157480315,
  'auprc': 0.8530475776210468,
  'acc': 0.7450980392156863,
  'eval_loss': 0.5115815047174692},
 array([[ 2.00952768e-01, -7.84827471e-02],
        [ 2.21466973e-01, -4.04441029e-01],
        [ 2.13100046e-01, -3.49926859e-01],
        [ 8.39725509e-02, -2.24679440e-01],
        [-8.74745786e-01,  1.02287722e+00],
        [ 1.92381114e-01, -1.94868386e-01],
        [ 5.66268444e-01, -7.74571419e-01],
        [-3.68729323e-01,  3.39812398e-01],
        [ 8.40693831e-01, -7.93415010e-01],
        [ 5.82896531e-01, -7.60865808e-01],
        [ 1.65860668e-01, -3.26481193e-01],
        [-1.53569376e+00,  2.02870250e+00],
        [-2.29476482e-01, -1.58802439e-02],
        [-1.65728688e+00,  2.00827909e+00],
        [ 1.56394690e-01, -2.39274919e-01],
        [ 6.88504755e-01, -9.38124478e-01],
        [ 3.61681700e-01, -4.93699878e-01],
        [-1.40447652e+00,  2.22613215e+00],
   

# Model Selection with Validation data

In [17]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm

df = []

for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/roberta_" + str(num) + "/"
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        # print(checkpoint)
        model = ClassificationModel("roberta", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
        
df = pd.DataFrame(df)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


# Write File

In [20]:
df.to_json('outputs/roberta_eval_results.csv')

In [21]:
print('finished, yay')

finished, yay


In [30]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

,checkpoint,valid_acc,test_acc,config
156,outputs/roberta_14/checkpoint-160-epoch-8,0.799087,0.789346,"{'overwrite_output_dir': True, 'num_train_epoc..."
108,outputs/roberta_9/checkpoint-492-epoch-6,0.796043,0.794189,"{'overwrite_output_dir': True, 'num_train_epoc..."
111,outputs/roberta_10/checkpoint-123-epoch-3,0.796043,0.801453,"{'overwrite_output_dir': True, 'num_train_epoc..."
192,outputs/roberta_17/checkpoint-100-epoch-5,0.794521,0.796610,"{'overwrite_output_dir': True, 'num_train_epoc..."
153,outputs/roberta_13/checkpoint-205-epoch-5,0.792998,0.786925,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
245,outputs/roberta_22/checkpoint-41-epoch-1,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
246,outputs/roberta_22/checkpoint-82-epoch-2,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
252,outputs/roberta_22/checkpoint-205-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."
254,outputs/roberta_23/checkpoint-140-epoch-7,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [33]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.7990867579908676 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 32, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/roberta_14', 'warmup_ratio': 0.06, 'save_steps': 2000, 'manual_seed': 12345}

0.7960426179604262 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 64, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs